In [1]:

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

print("Project root added to sys.path")


Project root added to sys.path


In [2]:
from utils.data_loader import load_dataset
trainset, testset = load_dataset('cifar10', root='./data')
num_clients = 5

c:\Users\la7tim\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


100%|██████████| 170498071/170498071 [00:25<00:00, 6656921.35it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[INFO] Loaded dataset 'cifar10' with 50000 training samples and 10000 testing samples.


In [3]:
from utils.data_partition import iid_partition, non_iid_partition, dirichlet_partition, quantity_skew_partition, temporal_partition, hybrid_partition
iid_subsets = iid_partition(trainset, num_clients)
non_iid_subsets = non_iid_partition(trainset, num_clients, num_shards=10)
dirichlet_subsets = dirichlet_partition(trainset, num_clients, alpha=0.5)
quantity_subsets = quantity_skew_partition(trainset, num_clients, min_ratio=0.1, max_ratio=0.5)
temporal_subsets = temporal_partition(trainset, num_clients)
hybrid_subsets = hybrid_partition(trainset, num_clients, alpha=0.5, min_ratio=0.1, max_ratio=0.5)

# Organize partitions in a dictionary for easy iteration:
partitions = {
    "IID": iid_subsets,
    "Non-IID (Shards)": non_iid_subsets,
    "Dirichlet": dirichlet_subsets,
    "Quantity Skew": quantity_subsets,
    "Temporal": temporal_subsets,
    "Hybrid": hybrid_subsets
}

In [4]:
for key, subset in partitions.items():
    sizes = [len(ds) for ds in subset]
    print(f"{key} partitions sizes: {sizes}")

IID partitions sizes: [10000, 10000, 10000, 10000, 10000]
Non-IID (Shards) partitions sizes: [10000, 10000, 10000, 10000, 10000]
Dirichlet partitions sizes: [4826, 9903, 13669, 13173, 8429]
Quantity Skew partitions sizes: [12643, 10942, 4740, 14150, 7525]
Temporal partitions sizes: [10000, 10000, 10000, 10000, 10000]
Hybrid partitions sizes: [4739, 4739, 2611, 1466, 4202]


In [6]:
from clients.federated_training import federated_training

results = {}
rounds = 5

for strategy, subsets in partitions.items():
    print(f"\n--- Federated Training with {strategy} partitions ---")
    global_model, test_accs = federated_training(
        client_datasets=subsets,
        model_name='cifar10',
        testset=testset,
        rounds=rounds,
        device="cpu"
    )
    results[strategy] = test_accs

print("Federated Training CIFAR-10 Test Accuracies for each partition strategy:")
for strat, accs in results.items():
    print(f"{strat}: {accs}")



--- Federated Training with IID partitions ---

Round 1/5
Test Accuracy: 0.2331

Round 2/5
Test Accuracy: 0.1902

Round 3/5


KeyboardInterrupt: 